# (MBTI) Myers-Briggs Personality Type Prediction

* Extroversion vs. Introversion
    * I - 0
    * E - 1 
    
* Sensing vs. Intuition 
    * N - 0 
    * S - 1
    
* Thinking vs. Feeling
    * F - 0
    * T - 1
    
* Judging vs. Perceiving
    * P - 0
    * J - 1 

In [1]:
# importing dependencies here
import numpy as np
import pandas as pd

# visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# feature engineering
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("wordnet")
nltk.download("vader_lexicon")

# sentiment scoring
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# scikit
# vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# scaling to handle negative values (for Naive Bayes)
from sklearn.preprocessing import MinMaxScaler

# data stratifying and splitting
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

# algorithms/models
# from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import Normalizer
from imblearn.pipeline import make_pipeline as imb_make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

# model performance evaluation and selection
from sklearn.metrics import (
    classification_report,
    f1_score,
    accuracy_score,
    roc_auc_score,
)

# performance check
import time

from joblib import load

# code formatter
%load_ext nb_black

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eshom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eshom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\eshom\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eshom\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\eshom\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


<IPython.core.display.Javascript object>

In [2]:
# reading the final datasets
df = pd.read_csv("../data/test_data.csv")

<IPython.core.display.Javascript object>

In [3]:
# checking counts dataset
df.head(2)

,name,type,posts
0,Donald Trump,ESTP,My thoughts and prayers are with the @USMC cre...
1,Barack Obama,ENFJ,"Happy Hanukkah! Over these eight nights, we dr..."


<IPython.core.display.Javascript object>

In [4]:
def categorize_types(personality_data):

    personality_data["is_Extrovert"] = personality_data["type"].apply(
        lambda x: 1 if x[0] == "E" else 0
    )
    personality_data["is_Sensing"] = personality_data["type"].apply(
        lambda x: 1 if x[1] == "S" else 0
    )
    personality_data["is_Thinking"] = personality_data["type"].apply(
        lambda x: 1 if x[2] == "T" else 0
    )
    personality_data["is_Judging"] = personality_data["type"].apply(
        lambda x: 1 if x[3] == "J" else 0
    )

    # rearranging the dataframe columns
    personality_data = personality_data[
        ["type", "is_Extrovert", "is_Sensing", "is_Thinking", "is_Judging", "posts"]
    ]


#######################################################################################################3


def clean_posts(personality_data):

    # converting posts into lower case
    personality_data["clean_posts"] = personality_data["posts"].str.lower()

    # replacing ||| with space
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        re.compile(r"\|\|\|"), ""
    )

    # replacing urls with domain name
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        re.compile(r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*"),
        lambda match: match.group(2),
    )

    # dropping emails
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        re.compile(r"\S+@\S+"), ""
    )

    # dropping punctuations
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        re.compile(r"[^a-z\s]"), ""
    )

    # dropping MBTIs mentioned in the posts. There are quite a few mention of these types in these posts.
    mbti = personality_data["type"].unique()
    for type_word in mbti:
        personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
            type_word.lower(), ""
        )

    # tag_posts will be a list of 50 lists. need it for word stats (per post for each user)
    # replacing urls with domain name
    personality_data["tag_posts"] = personality_data["posts"].str.replace(
        re.compile(r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*"),
        lambda match: match.group(2),
    )
    # replacing ||| with space
    personality_data["tag_posts"] = [
        post for post in personality_data["tag_posts"].str.split("\|\|\|")
    ]


#################################################################################################################


def sentiment_score(personality_data):

    analyzer = SentimentIntensityAnalyzer()

    nlp_sentiment_score = []

    for post in personality_data["clean_posts"]:
        score = analyzer.polarity_scores(post)["compound"]
        nlp_sentiment_score.append(score)

    personality_data["compound_sentiment"] = nlp_sentiment_score


###############################################################################################################


def pos_tagging(personality_data):

    personality_data["tagged_words"] = personality_data["tag_posts"].apply(
        lambda x: [nltk.pos_tag(word_tokenize(line)) for line in x]
    )

    # grouping pos tags based on stanford list
    tags_dict = {
        "ADJ": ["JJ", "JJR", "JJS"],
        "ADP": ["EX", "TO"],
        "ADV": ["RB", "RBR", "RBS", "WRB"],
        "CONJ": ["CC", "IN"],
        "DET": ["DT", "PDT", "WDT"],
        "NOUN": ["NN", "NNS", "NNP", "NNPS"],
        "NUM": ["CD"],
        "PRT": ["RP"],
        "PRON": ["PRP", "PRP$", "WP", "WP$"],
        "VERB": ["MD", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
        ".": ["#", "$", "''", "(", ")", ",", ".", ":"],
        "X": ["FW", "LS", "UH"],
    }

    def stanford_tag(x, tag):
        tags_list = [len([y for y in line if y[1] in tags_dict[col]]) for line in x]
        return tags_list

    for col in tags_dict.keys():
        personality_data["S_" + col + "_med"] = personality_data["tagged_words"].apply(
            lambda x: np.median(stanford_tag(x, col))
        )
        personality_data["S_" + col + "_std"] = personality_data["tagged_words"].apply(
            lambda x: np.std(stanford_tag(x, col))
        )


###############################################################################################################


def get_counts(personality_data):
    def unique_words(s):
        unique = set(s.split(" "))
        return len(unique)

    def emojis(post):
        # does not include emojis made purely from symbols, only :word:
        emoji_count = 0
        words = post.split()
        for e in words:
            if "http" not in e:
                if e.count(":") == 2:
                    emoji_count += 1
        return emoji_count

    def colons(post):
        # Includes colons used in emojis
        colon_count = 0
        words = post.split()
        for e in words:
            if "http" not in e:
                colon_count += e.count(":")
        return colon_count

    personality_data["qm"] = personality_data["posts"].apply(lambda s: s.count("?"))
    personality_data["em"] = personality_data["posts"].apply(lambda s: s.count("!"))
    personality_data["colons"] = personality_data["posts"].apply(colons)
    personality_data["emojis"] = personality_data["posts"].apply(emojis)
    personality_data["word_count"] = personality_data["posts"].apply(
        lambda s: s.count(" ") + 1
    )
    personality_data["unique_words"] = personality_data["posts"].apply(unique_words)
    personality_data["avg_word_ct"] = personality_data["word_count"].apply(
        lambda s: s / 50
    )
    personality_data["post_length_var"] = personality_data["posts"].apply(
        lambda x: np.var([len(post.split()) for post in x.split("|||")])
    )
    #     personality_data["med_char"] = personality_data["tagged_words"].apply(
    #         lambda x: np.median([len(i) for i in x]))
    #     personality_data["med_word"] = personality_data["tagged_words"].apply(
    #         lambda x: np.median([len(i.split()) for i in x]))
    personality_data["upper"] = personality_data["posts"].apply(
        lambda x: len([x for x in x.split() if x.isupper()])
    )
    personality_data["link_count"] = personality_data["posts"].apply(
        lambda s: s.count("http")
    )
    ellipses_count = [
        len(re.findall(r"\.\.\.\ ", posts)) for posts in personality_data["posts"]
    ]
    personality_data["ellipses"] = ellipses_count
    personality_data["img_count"] = [
        len(re.findall(r"(\.jpg)|(\.jpeg)|(\.gif)|(\.png)", post))
        for post in personality_data["posts"]
    ]


#################################################################################################################


def vectorize(personality_data):

    tfidf_vectorizer = TfidfVectorizer(
        min_df=0.05, max_df=0.85, analyzer="word", ngram_range=(1, 2)
    )
    tfidf_words = tfidf_vectorizer.fit_transform(personality_data["clean_posts"])

    tfidf_vectorized_data = pd.DataFrame(
        data=tfidf_words.toarray(), columns=tfidf_vectorizer.get_feature_names()
    )
    return tfidf_vectorized_data

<IPython.core.display.Javascript object>

In [5]:
def prep_data(personality_data):

    t = time.time()

    categorize_types(personality_data)

    clean_posts(personality_data)

    sentiment_score(personality_data)

    pos_tagging(personality_data)

    get_counts(personality_data)

    tfidf_vectorized_data = vectorize(personality_data)

    features = personality_data[
        [
            "compound_sentiment",
            "S_ADJ_med",
            "S_ADJ_std",
            "S_ADP_med",
            "S_ADP_std",
            "S_ADV_med",
            "S_ADV_std",
            "S_CONJ_med",
            "S_CONJ_std",
            "S_DET_med",
            "S_DET_std",
            "S_NOUN_med",
            "S_NOUN_std",
            "S_NUM_med",
            "S_NUM_std",
            "S_PRT_med",
            "S_PRT_std",
            "S_PRON_med",
            "S_PRON_std",
            "S_VERB_med",
            "S_VERB_std",
            "qm",
            "em",
            "colons",
            "emojis",
            "word_count",
            "unique_words",
            "avg_word_ct",
            "post_length_var",
            #         "med_char",
            #         "med_word",
            "upper",
            "link_count",
            "ellipses",
            "img_count",
        ]
    ]

    X = pd.concat([features, tfidf_vectorized_data], axis=1)
    y = personality_data.iloc[:, 3:7]

    print(f"Total Preprocessing Time: {time.time()-t} seconds")

    return X, y

<IPython.core.display.Javascript object>

In [6]:
X, y = prep_data(df)

Total Preprocessing Time: 1.0182762145996094 seconds


<IPython.core.display.Javascript object>

In [7]:
X

,compound_sentiment,S_ADJ_med,S_ADJ_std,S_ADP_med,S_ADP_std,S_ADV_med,S_ADV_std,S_CONJ_med,S_CONJ_std,S_DET_med,...,youve been,youxkarla,youxkarla will,yzy,yzy ssn,zainraz,zainraz erdayastronaut,zero,zero guarantees,zero money
0,0.9986,9.0,5.202089,1.0,3.154460,6.0,2.671292,14.0,10.788654,9.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.04458,0.02229,0.02229
1,0.9996,53.5,5.500000,20.0,2.000000,17.5,4.500000,65.5,5.500000,48.0,...,0.019627,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.9895,1.0,2.645751,0.0,0.400000,0.0,0.663325,2.0,2.647640,1.0,...,0.000000,0.000000,0.000000,0.064439,0.064439,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.9998,30.5,3.500000,12.5,7.500000,14.0,1.000000,49.5,6.500000,33.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.9973,1.0,1.690628,0.0,0.605582,0.0,1.466707,1.0,1.111888,1.0,...,0.000000,0.051198,0.051198,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
5,0.9976,1.0,1.082809,0.0,0.201848,0.0,1.087815,0.0,1.164201,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
6,0.9986,4.0,10.754270,1.0,2.806518,2.0,7.410370,6.0,9.775252,4.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
7,0.9976,10.5,11.368817,4.0,1.920286,6.0,2.487469,9.5,14.686303,5.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
8,0.9963,10.0,13.440238,1.0,5.351635,2.0,6.499231,12.0,23.465720,6.0,...,0.027623,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.9934,1.0,1.612198,0.0,0.805086,0.0,1.275835,1.0,1.988539,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.03495,0.03495,0.00000,0.00000,0.00000


<IPython.core.display.Javascript object>

In [8]:
y

,is_Extrovert,is_Sensing,is_Thinking,is_Judging
0,1,1,1,0
1,1,0,0,1
2,0,1,0,0
3,0,0,1,1
4,1,1,0,0
5,0,0,0,0
6,1,0,0,0
7,0,1,0,1
8,0,0,0,1
9,0,0,1,1


<IPython.core.display.Javascript object>

In [9]:
df.head()

,name,type,posts,is_Extrovert,is_Sensing,is_Thinking,is_Judging,clean_posts,tag_posts,compound_sentiment,...,colons,emojis,word_count,unique_words,avg_word_ct,post_length_var,upper,link_count,ellipses,img_count
0,Donald Trump,ESTP,My thoughts and prayers are with the @USMC cre...,1,1,1,0,my thoughts and prayers are with the usmc crew...,[My thoughts and prayers are with the @USMC cr...,0.9986,...,1,0,917,514,18.34,37.5700,34,42,0,0
1,Barack Obama,ENFJ,"Happy Hanukkah! Over these eight nights, we dr...",1,0,0,1,happy hanukkah over these eight nights we draw...,"[Happy Hanukkah! Over these eight nights, we d...",0.9996,...,4,0,991,574,19.82,7.0400,8,50,0,0
2,Kanye West,ISFP,@jarrodspector @TheCherShow the dynamics of Ch...,0,1,0,0,jarrodspector thechershow the dynamics of cher...,[@jarrodspector @TheCherShow the dynamics of C...,0.9895,...,0,0,204,164,4.08,43.5604,4,42,1,0
3,Arnold Schwarzenegger,INTJ,Fantastic to spend some time with you teaming ...,0,0,1,1,fantastic to spend some time with you teaming ...,[Fantastic to spend some time with you teaming...,0.9998,...,4,0,752,411,15.04,35.1396,14,49,0,0
4,Justin Bieber,ESFP,All love over here Aaron. You got my support||...,1,1,0,0,all love over here aaron you got my supportaar...,"[All love over here Aaron. You got my support,...",0.9973,...,2,0,293,211,5.86,39.3056,13,32,1,0


<IPython.core.display.Javascript object>

### Modelling

In [10]:
def model(model, X, target):

    mbti_type = {
        "is_Extrovert": "Extrovert vs Introvert",
        "is_Sensing": "Sensing vs Intuition",
        "is_Thinking": "Thinking vs Feeling",
        "is_Judging": "Judging vs Perceiving",
    }

    for col in target.columns:

        print(f"\n{mbti_type[col]}")
        print(target[col])
        y = target[col]

        y_pred = model.predict(X)

        preds = model.predict_proba(X)[:, 1]  # returns the probablity of class 1
        auc = roc_auc_score(y_test, preds)
        acc = accuracy_score(y_test, preds.round())
        f1 = f1_score(y_test, preds.round())

        auc_list.append(auc)
        acc_list.append(acc)
        f1_list.append(f1)

        print(f"Avg AUC: {auc:.2f}, Avg Accuracy: {acc:.2f}, Avg F1: {f1:.2f}")

        print(classification_report(y_test, y_pred))

    print(f"\nTime Taken: {time.time()-t:.2f} seconds")

<IPython.core.display.Javascript object>

In [11]:
X.shape

(11, 6836)

<IPython.core.display.Javascript object>

In [12]:
y.shape

(11, 4)

<IPython.core.display.Javascript object>

In [13]:
clf = load("clf.joblib")
model(clf, X, y)


Extrovert vs Introvert
0     1
1     1
2     0
3     0
4     1
5     0
6     1
7     0
8     0
9     0
10    0
Name: is_Extrovert, dtype: int64


ValueError: operands could not be broadcast together with shapes (11,6836) (1504,) (11,6836) 

<IPython.core.display.Javascript object>